In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
def draw_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    y = rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]
    plt.plot(y[0],y[1], label = "spring origin {0:.1f}, {1:.1f}".format(start[0],start[1]))
    plt.xlim([-50,50])
    plt.ylim([-50,50])

In [12]:
def get_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    return rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]

In [4]:
spring = get_spring((-10.,-30.),(-30.,30.),10.,size = 5)
plt.plot(spring[0],spring[1])
plt.xlim([-50,50])
plt.ylim([-50,50])
plt.show()

In [9]:
plt.figure(figsize=(9,9))
draw_spring([-10.,-30.],[-30.,30.],10.,size = 5)
draw_spring([-20,20],[20,20],4,size = 2)
draw_spring([-30,30],[-20,20],10)

draw_spring([-20,0],[20,0],40,size = 10)
draw_spring([-20,30],[20,30],40,size = 4)
plt.legend(loc = 'upper left')      
plt.show()

In [6]:
class Spring:
    def __init__(self, start, end, count, size):
        self.start = start
        self.end = end
        self.count = count
        self.size = size
        
    def update_positions(self,start,end):
        self.start = start
        self.end = end
    
    def redraw(self):
        draw_spring(self.start,self.end, self.count,self.size)

In [15]:
import numpy as np
import matplotlib.pyplot as plt

class DraggableRectangle:
    def __init__(self, rect, springs):
        self.rect = rect
        self.press = None
        self.springs = springs

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print 'event contains', self.rect.xy
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        #print 'x0=%f, xpress=%f, event.xdata=%f, dx=%f, x0+dx=%f'%(x0, xpress, event.xdata, dx, x0+dx)
        self.rect.xy = (x0+dx,y0+dy)
        for spring in self.springs:
            y = get_spring((0.0,0.0),self.rect.xy,8,size=2.)
            spring.set_ydata(y[1])
            spring.set_xdata(y[0])
        plt.draw()


    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        plt.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

        
import matplotlib.patches as patches

fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal')
rekts = ax1.add_patch(
    patches.Rectangle(
        (0.1, 0.1),   # (x,y)
        0.5,          # width
        0.5,          # height
    )
)

y = get_spring((0.0,0.0),(0.4,0.4),8,size=2.)
spring, = ax1.plot(y[0],y[1])


drs = []
for rect in ax1.patches:
    dr = DraggableRectangle(rect,[spring])
    dr.connect()
    drs.append(dr)
    
plt.xlim((-20,20))
plt.ylim((-20,20))

plt.show()

event contains (0.1, 0.1)
event contains (0.1, 0.1)
event contains (19.68333333333333, 19.370833333333334)
event contains (19.68333333333333, 19.370833333333334)
event contains (16.87083333333333, -17.295833333333331)
event contains (16.974999999999994, 10.204166666666666)
event contains (10.099999999999994, 13.537499999999994)
event contains (16.349999999999994, 16.974999999999994)
event contains (12.599999999999994, 14.162499999999994)
event contains (14.37083333333333, 14.474999999999994)
event contains (-12.399999999999999, -9.6916666666666735)
event contains (8.9541666666666622, -11.462500000000007)
event contains (13.849999999999998, 2.1833333333333247)
